Using SQLAlchemy to Talk to a Database
=====================
SqlAlchemy helps you use a database to store and retrieve information from python.  It abstracts the specific storage engine from te way you use it - so it doesn't care if you end up using MySQL, SQLite, or whatever else. In addition, you can use core and the object-relational mapper (ORM) to avoid writing any SQL at all.  The [SQLAlchemy homepage](http://www.sqlalchemy.org/) has lots of good examples and full documentation.

In [71]:
from sqlalchemy import * 
import json, ast, unittest, datetime, mediacloud

In [188]:
mc = mediacloud.api.MediaCloud('0a65d2b3a0780cf15a40ca6914fe2dfeed17257e30b440c0ddffdfbc179635cd')

wordSearch = ['Christmas','Hell','Pants on fire','angry' ]
wordList = []
for index in range(len(wordSearch)):
    print wordSearch[index]
    wordsM = mc.sentenceCount( wordSearch[index],  solr_filter=[mc.publish_date_query( datetime.date( 2016, 7, 1), datetime.date( 2016, 12, 12) ), 'tags_id_media:1' ] )
    wordList.append(wordsM.copy())
                
print wordList

Christmas
Hell
Pants on fire
angry
[{u'count': 59609}, {u'count': 21914}, {u'count': 7851027}, {u'count': 33117}]


## Basic SQL Generation
The core library generates SQL for you.  Read more about it on their [expression language tutorial page](http://docs.sqlalchemy.org/en/rel_1_0/core/index.html). Below are some basic examples.

In [187]:

num_Mentions1 =  wordList[1]['count']
numMentStr1 = str(num_Mentions1)  

print num_Mentions1


21914


### Creating a Table
Read more about [defining and creating tables](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#define-and-create-tables).

In [146]:

# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:',echo=True) # just save the db in memory for now (ie. not on disk)
metadata = MetaData()
# define a table to use
queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keywords', String(400), nullable=False),
    Column('mentions', Integer,autoincrement=True),      
    Column('timestamp', DateTime, default=datetime.datetime.now),
)

metadata.create_all(engine) # and create the tables in the database


2016-12-14 17:50:20,599 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-12-14 17:50:20,600 INFO sqlalchemy.engine.base.Engine ()
2016-12-14 17:50:20,601 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-12-14 17:50:20,601 INFO sqlalchemy.engine.base.Engine ()
2016-12-14 17:50:20,602 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("queries")
2016-12-14 17:50:20,603 INFO sqlalchemy.engine.base.Engine ()
2016-12-14 17:50:20,604 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE queries (
	id INTEGER NOT NULL, 
	keywords VARCHAR(400) NOT NULL, 
	mentions INTEGER, 
	timestamp DATETIME, 
	PRIMARY KEY (id)
)


2016-12-14 17:50:20,604 INFO sqlalchemy.engine.base.Engine ()
2016-12-14 17:50:20,605 INFO sqlalchemy.engine.base.Engine COMMIT


### Inserting Data
Read more about generating [SQL insert statements](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#insert-expressions).

In [147]:
insert_stmt = queries.insert()
str(insert_stmt) # see an example of what this will do

'INSERT INTO queries (id, keywords, mentions, timestamp) VALUES (:id, :keywords, :mentions, :timestamp)'

In [150]:
insert_stmt = queries.insert().values(keywords= wordSearch,mentions= num_Mentions) )
str(insert_stmt)

'INSERT INTO queries (keywords, mentions, timestamp) VALUES (:keywords, :mentions, :timestamp)'

In [151]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

2016-12-14 17:51:10,132 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, mentions, timestamp) VALUES (?, ?, ?)
2016-12-14 17:51:10,133 INFO sqlalchemy.engine.base.Engine ('Christmas', 59609, '2016-12-14 17:51:10.132160')
2016-12-14 17:51:10,134 INFO sqlalchemy.engine.base.Engine COMMIT


[2]

In [152]:
insert_stmt = queries.insert().values(keywords= wordSearch2,mentions= num_Mentions2 )
str(insert_stmt)

'INSERT INTO queries (keywords, mentions, timestamp) VALUES (:keywords, :mentions, :timestamp)'

In [153]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

2016-12-14 17:51:51,174 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, mentions, timestamp) VALUES (?, ?, ?)
2016-12-14 17:51:51,175 INFO sqlalchemy.engine.base.Engine ('Hell', 21914, '2016-12-14 17:51:51.174611')
2016-12-14 17:51:51,176 INFO sqlalchemy.engine.base.Engine COMMIT


[3]

In [189]:
insert_stmt = queries.insert()
str(insert_stmt) # see an example of what this will do
for index in range(len(wordSearch)):
    insert_stmt = queries.insert().values(keywords= wordSearch[index],mentions= wordList[1]['count'] )
    str(insert_stmt)
    db_conn = engine.connect()
    result = db_conn.execute(insert_stmt)
    result.inserted_primary_key # print out the primary key it was assigned

2016-12-14 18:34:34,152 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, mentions, timestamp) VALUES (?, ?, ?)
2016-12-14 18:34:34,153 INFO sqlalchemy.engine.base.Engine ('Christmas', 21914, '2016-12-14 18:34:34.152721')
2016-12-14 18:34:34,154 INFO sqlalchemy.engine.base.Engine COMMIT
2016-12-14 18:34:34,155 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, mentions, timestamp) VALUES (?, ?, ?)
2016-12-14 18:34:34,155 INFO sqlalchemy.engine.base.Engine ('Hell', 21914, '2016-12-14 18:34:34.155030')
2016-12-14 18:34:34,156 INFO sqlalchemy.engine.base.Engine COMMIT
2016-12-14 18:34:34,157 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, mentions, timestamp) VALUES (?, ?, ?)
2016-12-14 18:34:34,158 INFO sqlalchemy.engine.base.Engine ('Pants on fire', 21914, '2016-12-14 18:34:34.157561')
2016-12-14 18:34:34,158 INFO sqlalchemy.engine.base.Engine COMMIT
2016-12-14 18:34:34,159 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords,

In [191]:
result = db_conn.execute(insert_stmt)


2016-12-14 18:35:16,923 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, mentions, timestamp) VALUES (?, ?, ?)
2016-12-14 18:35:16,924 INFO sqlalchemy.engine.base.Engine ('angry', 21914, '2016-12-14 18:35:16.923765')
2016-12-14 18:35:16,925 INFO sqlalchemy.engine.base.Engine COMMIT


### Retrieving Data
Read more about using [SQL select statments](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#selecting).

In [192]:
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    print row[2]

2016-12-14 18:35:32,360 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.mentions, queries.timestamp 
FROM queries
2016-12-14 18:35:32,361 INFO sqlalchemy.engine.base.Engine ()
59609
59609
21914
21914
21914
21914
21914
21914


In [162]:
select_stmt = select([queries]).where(queries.c.id==2)
for row in db_conn.execute(select_stmt):
    print row

2016-12-14 18:03:40,528 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.mentions, queries.timestamp 
FROM queries 
WHERE queries.id = ?
2016-12-14 18:03:40,529 INFO sqlalchemy.engine.base.Engine (2,)
(2, u'Christmas', 59609, datetime.datetime(2016, 12, 14, 17, 51, 10, 132160))


In [161]:
select_stmt = select([queries]).where(queries.c.keywords.like('C%'))
for row in db_conn.execute(select_stmt):
    print row

2016-12-14 18:03:06,790 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.mentions, queries.timestamp 
FROM queries 
WHERE queries.keywords LIKE ?
2016-12-14 18:03:06,790 INFO sqlalchemy.engine.base.Engine ('C%',)
(2, u'Christmas', 59609, datetime.datetime(2016, 12, 14, 17, 51, 10, 132160))


## ORM
You can use their ORM library to handle the translation into full-fledged python objects.  This can help you build the Model for you [MVC](https://en.wikipedia.org/wiki/Model–view–controller) solution.

In [84]:
import datetime
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
Base = declarative_base()

### Creating a class mapping
Read more about [creating a mapping](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#declare-a-mapping).

In [85]:
class Query(Base):
    __tablename__ = 'queries'
    id = Column(Integer, primary_key=True)
    keywords = Column(String(400))
    timestamp = Column(DateTime,default=datetime.datetime.now)
    def __repr__(self):
        return "<Query(keywords='%s')>" % (self.keywords)
Query.__table__

Table('queries', MetaData(bind=None), Column('id', Integer(), table=<queries>, primary_key=True, nullable=False), Column('keywords', String(length=400), table=<queries>), Column('timestamp', DateTime(), table=<queries>, default=ColumnDefault(<function now at 0x1120d2938>)), schema=None)

### Creating a connection and session
Read more about [creating this stuff](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#creating-a-session).

In [ ]:
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
my_session = Session()

### Inserting Data
Read more about [inserting data with an ORM](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#adding-new-objects).

In [ ]:
query = Query(keywords="iguana")
query.keywords

In [ ]:
my_session.add(query)
my_session.commit()
query.id

### Retrieving Data
Read more about [retrieving data from the db](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#querying) via an ORM class.

In [ ]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

In [ ]:
query1 = Query(keywords="robot")
query2 = Query(keywords="puppy")
my_session.add_all([query1,query2])
my_session.commit()

In [ ]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

In [ ]:
for q in my_session.query(Query).filter(Query.keywords.like('r%')):
    print q